### Laion 2b es4.5

In [3]:
import datasets
from datasets import load_dataset

In [4]:
ds = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_2b_en_as4.5plus")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-94e46530464c81b4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
type(ds['train'])

datasets.arrow_dataset.Dataset

In [12]:
len(ds['train'])

21440657

In [10]:
ds['train'].column_names

['URL',
 'TEXT',
 'WIDTH',
 'HEIGHT',
 'similarity',
 'punsafe',
 'pwatermark',
 'AESTHETIC_SCORE',
 'hash',
 '__index_level_0__']

In [11]:
sample0 = ds['train'].__getitem__(0)

{'URL': 'https://img1.etsystatic.com/186/0/5531524/il_340x270.1310915051_5kax.jpg',
 'TEXT': 'Vintage Green Glass Swag Hanging Lamp Mid Century',
 'WIDTH': 340.0,
 'HEIGHT': 270.0,
 'similarity': 0.3491742014884949,
 'punsafe': 8.991600225272123e-06,
 'pwatermark': 0.14151343703269958,
 'AESTHETIC_SCORE': 4.751741409301758,
 'hash': 6170073934346815248,
 '__index_level_0__': 0}

### Laion-art

In [15]:
art = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_art")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-d61c732c98ca6f56/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
art['train'].__getitem__(0)

{'URL': 'https://www.advocate-art.com/system/ART/Modules/Application/Images/Image/images/000/011/121/artistique_half/VML175107.jpg?6f71107bead7921d03fa7dc3e4ac4b9a8f24dfd3d823d512d7618c06e4059513',
 'TEXT': 'Christmas Shopping Copy',
 'WIDTH': 850,
 'HEIGHT': 850,
 'similarity': 0.2666246294975281,
 'LANGUAGE': 'nolang',
 'hash': -3604776403351267688,
 'pwatermark': 0.0396263524889946,
 'punsafe': 0.00027811527252197266,
 'aesthetic': 8.352225303649902}

In [17]:
len(art['train'])

8070941

In [25]:
8070941 * 0.1

807094.1000000001

### Filtering and Download

In [28]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import rand

#def filter_parquet(data_path_or_dir, width=None, height=None):
data_path_or_dir='/home/yx/datasets/diffusion/laion_art'
filter_width=512
filter_height=512
num_repartitions = 1
output_dir = '/data3/datasets/laion_art_filtered'

spark = SparkSession.builder.config("spark.driver.memory", "16G") .master("local[16]").appName('spark-stats').getOrCreate() 
df = spark.read.parquet(data_path_or_dir)



23/07/13 02:52:57 WARN Utils: Your hostname, zhengwei03 resolves to a loopback address: 127.0.0.1; using 10.90.53.110 instead (on interface enp98s0f1)
23/07/13 02:52:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 02:52:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/root/miniconda3/envs/ms20_py37/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [43]:
df.schema

StructType([StructField('URL', StringType(), True), StructField('TEXT', StringType(), True), StructField('WIDTH', IntegerType(), True), StructField('HEIGHT', IntegerType(), True), StructField('similarity', DoubleType(), True), StructField('LANGUAGE', StringType(), True), StructField('hash', LongType(), True), StructField('pwatermark', FloatType(), True), StructField('punsafe', FloatType(), True), StructField('aesthetic', FloatType(), True)])

In [36]:
df.printSchema()

root
 |-- URL: string (nullable = true)
 |-- TEXT: string (nullable = true)
 |-- WIDTH: integer (nullable = true)
 |-- HEIGHT: integer (nullable = true)
 |-- similarity: double (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- hash: long (nullable = true)
 |-- pwatermark: float (nullable = true)
 |-- punsafe: float (nullable = true)
 |-- aesthetic: float (nullable = true)



In [38]:
df.show()

+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+------------+---------+
|                 URL|                TEXT|WIDTH|HEIGHT|         similarity|LANGUAGE|                hash| pwatermark|     punsafe|aesthetic|
+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+------------+---------+
|https://www.advoc...|Christmas Shoppin...|  850|   850| 0.2666246294975281|  nolang|-3604776403351267688|0.039626352|2.7811527E-4| 8.352225|
|https://img.freep...|Cremiger hüttenkä...|  626|   415| 0.2779805064201355|      de|  571058205933102479| 0.06443764|0.0011278689| 8.013723|
|https://zakarpat....|A. Landovska Brea...|  300|   344|0.28194114565849304|      sv|-8455279219565229056|0.023449354|2.2380891E-5|8.0435295|
|https://www.povar...|Рецепт: Воздушные...|  330|   220| 0.2608181834220886|      ru|-3137830403123787972| 0.33741534|0.0047357082| 8.149608|
|http:

In [44]:
df.count()

8070941

In [47]:
"WIDTH" in df.schema.names

True

In [57]:
if "WIDTH" in df.schema.names:
    df = df.filter((df.WIDTH >= filter_width) & (df.HEIGHT >= filter_height) & (df.LANGUAGE == 'en'))
else:
    df = df.filter((df.width >= filter_width) & (df.height >= filter_height) & (df.language == 'en'))
df = df.orderBy(rand()) # this line is important to have a shuffled dataset
df.repartition(num_repartitions).write.parquet(output_dir)

In [58]:
df.count()

947191

In [59]:
947191/8070941

0.11735818661045844

In [61]:
df.show(10)

+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+-------------+---------+
|                 URL|                TEXT|WIDTH|HEIGHT|         similarity|LANGUAGE|                hash| pwatermark|      punsafe|aesthetic|
+--------------------+--------------------+-----+------+-------------------+--------+--------------------+-----------+-------------+---------+
|https://hummerlim...|Rolls Royce Phant...|  650|   520|0.34669220447540283|      en| 5889462542762778472| 0.28046012| 4.2948127E-4| 8.052956|
|https://www.carte...|Carters Steam Fai...| 1024|   576| 0.3125453293323517|      en| 2673312839296505774| 0.10378556|1.07132855E-5| 8.449714|
|https://cdn.shopi...|Watermelon One Pi...| 2048|  2048| 0.3144088089466095|      en|-7418655568329552979| 0.39337602| 0.0045395494| 8.104291|
|https://www.digsd...|Awesome Midsummer...|  595|   752|0.33127641677856445|      en| 5427967376058518284| 0.04007358|  2.413094E-4|  8.30234|

In [49]:
# no lang filter
df.count()

2378491

In [50]:
keep_ratio = 2378491/8070941
keep_ratio

0.2946981027367193

In [51]:
df.schema.names

['URL',
 'TEXT',
 'WIDTH',
 'HEIGHT',
 'similarity',
 'LANGUAGE',
 'hash',
 'pwatermark',
 'punsafe',
 'aesthetic']

## LAION 2b-en ori

In [19]:
ori  = load_dataset("parquet", data_dir="/home/yx/datasets/diffusion/laion_2b_en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/default-d308bf0fa364f6e6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
len(ori['train'])

18141938

In [21]:
ori

DatasetDict({
    train: Dataset({
        features: ['SAMPLE_ID', 'URL', 'TEXT', 'HEIGHT', 'WIDTH', 'LICENSE', 'NSFW', 'similarity'],
        num_rows: 18141938
    })
})

In [23]:
ori['train'].__getitem__(0)

{'SAMPLE_ID': 2641080021034,
 'URL': 'https://cdn.shopify.com/s/files/1/0017/3621/2538/products/blue-beach-umbrellas-point-of-rocks-crescent-beach-siesta-key-shawn-mcloughlin_32d72f5b-5e55-42f9-bfcf-d6fa8d239beb_300x300.jpg?v=1524171284',
 'TEXT': 'Blue Beach Umbrellas, Point Of Rocks, Crescent Beach, Siesta Key - Spiral Notebook',
 'HEIGHT': 231,
 'WIDTH': 300,
 'LICENSE': '?',
 'NSFW': 'UNLIKELY',
 'similarity': 0.3955616354942322}

In [62]:
import os

os.listdir(".")

['fusion_result.json',
 'rank_0',
 "':w",
 'test_load_laion.ipynb',
 'rank_1',
 'tools',
 'arch_params.txt',
 'download_laion_imgs.sh',
 'demo.md',
 'utils',
 'output_sd1x',
 'schedulers.md',
 'params_ori.txt',
 'train_text_to_image.py',
 'laion_check_and_convert.py',
 'laion_filter_metadata.py',
 '.laion_train.md.swp',
 'nohup.out',
 '.laion_filter_metadata.py.swp',
 'bpe_simple_vocab_16e6.txt.gz',
 'test_ge.py',
 'infer_ge.sh',
 'Untitled.ipynb',
 '.laion_check_and_convert.py.swp',
 'arch.txt',
 'demo_res.ipynb',
 'requirements.txt',
 'test_lora_ori_net.ckpt',
 '.ipynb_checkpoints',
 'output',
 'test_lora_tp_after_ft.ckpt',
 'train_autoencoder.py',
 'lora_finetune.md',
 'test_load_laion.py',
 '.download_laion_imgs.sh.swp',
 'scripts',
 'kernel_meta_temp_f87b28a10d8fafc3e16fe1b9ae8d913dadc9bee95d1a8140e304f8db9e4d8086',
 'ckpt_ana_res.txt',
 'models',
 'laion_train.md',
 'ldm',
 'bak',
 '__pycache__',
 'analyze_ckpt.py',
 'convert_dataset.py',
 'laion-art',
 ';',
 'arch_lora.txt',
 't